**Задание 1**  
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

Наименование вакансии.
Предлагаемую зарплату (отдельно минимальную и максимальную).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
import json
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

main_link_hh = 'https://hh.ru'
main_link_sj = 'https://www.superjob.ru'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

search_text = input('Введите ключевое слово для поиска вакансий: ')

# Параметры для формирования GET запросов
# Поиск осуществляется по вакансиям в Москве

params_hh = {
        'clusters': 'true',  
        'enable_snippets': 'true',  
        'text': search_text,
        'L_save_area': 'true',
        'area': '1', 
        'from': 'cluster_area',
        'showClusters': 'false'
    }

params_sj = {
        'keywords': search_text,  
        'geo[t][0]': 4
    }

Введите ключевое слово для поиска вакансий: python


In [2]:
# Сбор информации о вакансиях с площадки Head Hanter

# Определяем максимальное количество web страниц

html_hh = requests.get(main_link_hh + '/search/vacancy', params=params_hh, headers=headers)
soup_hh = bs(html_hh.text,'html.parser')
page_hh = soup_hh.find('div',attrs={'data-qa':'pager-block'}).getText()
max_page_hh = int(max(re.findall(r'\d+', page_hh)))

# Список вакансий

vacancies_hh = []

# Производим поиск вакансий по всем страницам с указанными параметрами результат добавляем в список vacancies_hh

for page in range(max_page_hh):
    params = {
        'clusters': 'true',  
        'enable_snippets': 'true',  
        'text': search_text,
        'L_save_area': 'true',
        'area': '1',
        'from': 'cluster_area',
        'showClusters': 'false',
        'page': page
    }

    html = requests.get(main_link_hh + '/search/vacancy', params=params, headers=headers)
    soup = bs(html.text,'html.parser')

    vacancies_block = soup.find('div',{'class':'vacancy-serp'})
    vacancies_list = vacancies_block.find_all('div',{'class':'vacancy-serp-item'}) 
    
    for vacancy in vacancies_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class':'bloko-link HH-LinkModifier'}).getText()
        vacancy_link = vacancy.find('a', {'class':'bloko-link HH-LinkModifier'})['href']
        employer_link = main_link_hh + vacancy.find('div', {'class':'vacancy-serp-item__meta-info'}).find('a')['href']
        
        vacancy_data['vacancy_name'] = vacancy_name
        vacancy_data['vacancy_link'] = vacancy_link
        vacancy_data['employer_link'] = employer_link

        salalary_data = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText()
        salary_list = salalary_data.replace(u'\xa0', '').replace('-', ' ').split()

        if 'от' in salary_list:
            vacancy_data['salary_min'] = int(salary_list[1])
            vacancy_data['salary_max'] = None
            vacancy_data['salary_unit'] = salary_list.pop()
        elif 'до' in salary_list:
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = int(salary_list[1])
            vacancy_data['salary_unit'] = salary_list.pop()
        elif len(salary_list) > 2:
            vacancy_data['salary_min'] = int(salary_list[0])
            vacancy_data['salary_max'] = int(salary_list[1])
            vacancy_data['salary_unit'] = salary_list.pop()
        else:
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = None
            vacancy_data['salary_unit'] = None

        vacancies_hh.append(vacancy_data)

print(f'По вашему запросу найдено {len(vacancies_hh)} вакансий на {max_page_hh} страницах')

По вашему запросу найдено 2000 вакансий на 40 страницах


In [3]:
# Сбор информации о вакансиях с площадки Super job

# Определяем максимальное количество web страниц

html_sj = requests.get(main_link_sj + '/vacancy/search/', params=params_sj, headers=headers)
soup_sj = bs(html_sj.text,'html.parser')
page_sj = soup_sj.find('div',{'class':'_3zucV L1p51 undefined _1Fty7 _2tD21 _3SGgo'}).getText()
max_page_sj = int(max([n for n in re.findall(r'\d+', page_sj)[0]]))

# Список вакансий

vacancies_sj = []

# Производим поиск вакансий по всем страницам с указанными параметрами результат добавляем в список vacancies_sj

for page in range(max_page_sj):
    params = {
        'keywords': search_text,  
        'geo[t][0]': 4,
        'page': page
    }

    html = requests.get(main_link_sj + '/vacancy/search/', params=params_sj, headers=headers)
    soup = bs(html.text,'html.parser')

    vacancies_block = soup.find('div',{'class':'_1Ttd8'})
    vacancies_list = vacancies_block.find_all('div',{'class':'Fo44F QiY08 LvoDO'})

        
    for vacancy in vacancies_list:
        vacancy_data = {} 
        vacancy_name = vacancy.find('div',{'class':'_3mfro PlM3e _2JVkc _3LJqf'}).getText()
        vacancy_link = main_link_sj + vacancy.find('div',{'class':'_3mfro PlM3e _2JVkc _3LJqf'}).find('a')['href']
        employer_link = main_link_sj + vacancy.find('div', {'class':['_3_eyK _3P0J7 _9_FPy']}).find('a')['href']

        vacancy_data['vacancy_name'] = vacancy_name
        vacancy_data['vacancy_link'] = vacancy_link
        vacancy_data['employer_link'] = employer_link

        salalary_data = vacancy.find('span', {'class':'_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}).getText()
        salary_string = salalary_data.replace(u'\xa0', '')


        if 'По' in salary_string:
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = None
            vacancy_data['salary_unit'] = None     

        elif '—' in salary_string:
            salary_list = re.findall(r'\d+', salary_string)
            salary_list.append(re.findall(r'руб', salary_string)[0])

            vacancy_data['salary_min'] = int(salary_list[0])
            vacancy_data['salary_max'] = int(salary_list[1])
            vacancy_data['salary_unit'] = salary_list.pop()

        elif 'до' in salary_string:
            salary_list = re.findall(r'\d+', salary_string)
            salary_list.append(re.findall(r'руб', salary_string)[0])

            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = int(salary_list[0])
            vacancy_data['salary_unit'] = salary_list.pop()  

        else:
            salary_list = re.findall(r'\d+', salary_string)
            salary_list.append(re.findall(r'руб', salary_string)[0])
            
            vacancy_data['salary_min'] = None
            vacancy_data['salary_max'] = int(salary_list[0])
            vacancy_data['salary_unit'] = salary_list.pop()

        vacancies_sj.append(vacancy_data)
        
print(f'По вашему запросу найдено {len(vacancies_sj)} вакансий на {max_page_sj} страницах')

По вашему запросу найдено 60 вакансий на 3 страницах


In [4]:
# Результаты поиска объеденим в датафрейм result_df

result_df = pd.DataFrame(vacancies_hh + vacancies_sj)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060 entries, 0 to 2059
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vacancy_name   2060 non-null   object 
 1   vacancy_link   2060 non-null   object 
 2   employer_link  2060 non-null   object 
 3   salary_min     381 non-null    float64
 4   salary_max     321 non-null    float64
 5   salary_unit    483 non-null    object 
dtypes: float64(2), object(4)
memory usage: 96.7+ KB


In [5]:
result_df.head(10)

,vacancy_name,vacancy_link,employer_link,salary_min,salary_max,salary_unit
0,Программист-разработчик (python),https://ekaterinburg.hh.ru/vacancy/37674249?qu...,https://hh.ru/employer/1776381,100000.0,180000.0,руб.
1,Machine Learning Developer,https://ekaterinburg.hh.ru/vacancy/37844601?qu...,https://hh.ru/employer/1455?dpt=hh-1455-ds,NaN,NaN,None
2,Junior Python (Django),https://ekaterinburg.hh.ru/vacancy/39110303?qu...,https://hh.ru/employer/771418,NaN,70000.0,руб.
3,Python-Разработчик (удаленно),https://ekaterinburg.hh.ru/vacancy/38624353?qu...,https://hh.ru/employer/2730772,NaN,250000.0,руб.
4,Python разработчик,https://ekaterinburg.hh.ru/vacancy/39085104?qu...,https://hh.ru/employer/2386651,100000.0,120000.0,руб.
5,Программист Python,https://ekaterinburg.hh.ru/vacancy/38660476?qu...,https://hh.ru/employer/4835563,120000.0,170000.0,руб.
6,"Python-разработчик (Django, DRF)",https://ekaterinburg.hh.ru/vacancy/39137779?qu...,https://hh.ru/employer/4596113,60000.0,150000.0,руб.
7,Программист Python,https://ekaterinburg.hh.ru/vacancy/39053902?qu...,https://hh.ru/employer/1517303,125000.0,180000.0,руб.
8,Разработчик Python,https://ekaterinburg.hh.ru/vacancy/39108864?qu...,https://hh.ru/employer/66123,150000.0,170000.0,руб.
9,Python - Software Engineer,https://ekaterinburg.hh.ru/vacancy/38715377?qu...,https://hh.ru/employer/4805547,2500.0,4000.0,USD


In [6]:
# Итоговый датафрейм сохраним в файл result_df.csv

result_df.to_csv('result_df.csv')